<a href="https://colab.research.google.com/github/woodRock/grokking-deep-learning/blob/main/chapter_6_building_your_first_deep_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np

np.random.seed(1)

def relu(x):
    return (x > 0) * x

def relu2deriv(output):
    return output > 0

dataset = "traffic"

X, y = None, None

if dataset == "XOR":
    X = np.array([[0,0],
              [0,1],
              [1,0],
              [1,1]])

    y = np.array([[0,1,1,0]]).T
elif dataset == "traffic":
    X = np.array([[1,0,1],
                [0,1,1],
                [0,0,1],
                [1,1,1]])

    y = np.array([[1,1,0,0]]).T

alpha = 0.2
input_dim = X.shape[1]
hidden_size = 4
num_hidden = 0
output_dim = y.shape[1]
epochs = 60

layers = []
first = 2 * np.random.random((input_dim, hidden_size)) - 1
second = 2 * np.random.random((hidden_size, output_dim)) - 1
hidden = [(2 * np.random.random((hidden_size, hidden_size)) - 1) for _ in range(num_hidden)]
layers.append(first)
layers.extend(hidden)
layers.append(second)

predictions = None

for iteration in range(epochs):
    loss = 0
    predictions = []
    for i in range(len(X)):
        layer = [0] * (len(layers) + 1)
        input, target = X[i:i+1], y[i:i+1]

        # Get the predictions.
        for i, _ in enumerate(range(len(layer))):
            if i == 0:
                layer[i] = input
            elif i == len(layer) - 1:
                layer[i] = np.dot(layer[i-1], layers[i-1])
            else:
                layer[i] = relu(np.dot(layer[i-1], layers[i-1]))

        prediction = layer[-1]
        predictions.append(prediction)

        loss += np.sum((prediction - target) ** 2)

        # Calculate the deltas
        deltas = [0] * (len(layers) + 1)
        for i in reversed(range(len(deltas))):
            if i == len(layer) - 1:
                deltas[i] = (prediction - target)
            else:
                deltas[i] = deltas[i+1].dot(layers[i].T) * relu2deriv(layer[i])

        # Update the weights.
        for i in reversed(range(len(deltas) - 1)):
            layers[i] -= alpha * layer[i].T.dot(deltas[i+1])

    if (iteration % 10 == 9):
        print(f"Error: {loss}")

print(f"Final prediction: {predictions}")

if dataset == "traffic":
    # For the traffic light problem with 60 epochs of training.
    assert loss == 1.5055622665134859e-05

Error: 0.6342311598444467
Error: 0.35838407676317513
Error: 0.0830183113303298
Error: 0.006467054957103705
Error: 0.0003292669000750734
Error: 1.5055622665134859e-05
Final prediction: [array([[1.]]), array([[0.99830926]]), array([[0.00349242]]), array([[0.]])]
